In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# Frontal vs Lateral (View) Model Testing
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
from sklearn.metrics import precision_score, recall_score, roc_auc_score
import numpy as np


model_weights = #input the path of the model weights pth file#

test_dict = #input the root directory of the test folder with two subfolders containing images#

num_classes = 2  


resnet50 = models.resnet50(weights=None)
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, num_classes)  


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50.to(device)
state_dict = torch.load(model_weights, map_location=device)
resnet50.load_state_dict(state_dict)
resnet50.eval()


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])


test_dataset = ImageFolder(root=test_dict, transform=transform)


test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0)


def get_predictions_and_probs(model, data_loader):
    predictions = []
    ground_truth = []
    probabilities = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  
            _, predicted = torch.max(outputs, 1) 
            
            predictions.extend(predicted.cpu().numpy())
            ground_truth.extend(labels.numpy())
            probabilities.extend(probs.cpu().numpy())

    return predictions, ground_truth, probabilities


test_predictions, test_ground_truth, test_probs = get_predictions_and_probs(resnet50, test_loader)


def calculate_accuracy(predictions, ground_truth):
    correct = sum([1 for p, gt in zip(predictions, ground_truth) if p == gt])
    total = len(ground_truth)
    accuracy = correct / total
    return accuracy


overall_accuracy = calculate_accuracy(test_predictions, test_ground_truth)


auroc = roc_auc_score(test_ground_truth, np.array(test_probs)[:, 1])


precision = precision_score(test_ground_truth, test_predictions)
recall = recall_score(test_ground_truth, test_predictions)


print("Overall Accuracy:", overall_accuracy)
print("AUROC:", auroc)
print("Precision:", precision)
print("Recall:", recall)


Overall Accuracy: 0.9945
AUROC: 0.9957900000000001
Precision: 0.9930209371884346
Recall: 0.996


In [8]:
# Normal vs Abnormal Model Testing

import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
from sklearn.metrics import precision_score, recall_score, roc_auc_score
import numpy as np
from PIL import Image

model_weights = #input the path of the model weights pth file#

test_dict = #input the root directory of the test folder with two subfolders containing images#


class CustomImageFolder(ImageFolder):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __getitem__(self, index):
        while True:
            try:
                return super().__getitem__(index)  
            except (OSError, ValueError) as e: 
                print(f"Error loading image at index {index}: {e}")
                index += 1  


num_classes = 2 


resnet50 = models.resnet50(weights=None)
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, num_classes) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50.to(device)
state_dict = torch.load(model_weights, map_location=device)
resnet50.load_state_dict(state_dict)
resnet50.eval()


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])


test_dataset = CustomImageFolder(root=test_dict, transform=transform)


test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0
                         
def get_predictions_and_probs(model, data_loader):
    predictions = []
    ground_truth = []
    probabilities = []

    with torch.no_grad():
        for inputs, labels in data_loader:

            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  
            _, predicted = torch.max(outputs, 1)  

            predictions.extend(predicted.cpu().numpy())
            ground_truth.extend(labels.numpy())
            probabilities.extend(probs.cpu().numpy())

    return predictions, ground_truth, probabilities


test_predictions, test_ground_truth, test_probs = get_predictions_and_probs(resnet50, test_loader)


def calculate_accuracy(predictions, ground_truth):
    correct = sum([1 for p, gt in zip(predictions, ground_truth) if p == gt])
    total = len(ground_truth)
    accuracy = correct / total
    return accuracy


overall_accuracy = calculate_accuracy(test_predictions, test_ground_truth)



auroc = roc_auc_score(test_ground_truth, np.array(test_probs)[:, 1])


precision = precision_score(test_ground_truth, test_predictions)
recall = recall_score(test_ground_truth, test_predictions)


print("Overall Accuracy:", overall_accuracy)
print("AUROC:", auroc)
print("Precision:", precision)
print("Recall:", recall)


Error loading image at index 5: broken data stream when reading image file
Overall Accuracy: 0.9921568627450981
AUROC: 0.9894
Precision: 0.996993987975952
Recall: 0.995
